In [10]:
import os
os.environ["PATH"] += os.pathsep + r"/Users/mmymacymac/Developer/Tools/ffmpeg-2025-08-18-git-0226b6fb2c-full_build/bin"

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True, # 청크별로 타임스탬프 변환
    chunk_length_s=10, # 입력 오디오 10초씩 나누기
    stride_length_s=2, # 2초씩 겹치도록 청크 나누기
)

sample = "../audio/calling01.mp3"

result = pipe(sample) # waveform 형태로 변환 bt FFMPEG

print(result)

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


{'text': ' Музыка. 상담사와 연결되었습니다. 어? 여보세요 안녕하세요 이름 말씀드리면 될까요? 네, 말씀해주세요. 네, 이름 말씀드리면 될까요? 네, 생년월일 말씀해주세요. 네, 11월 6일이요. 몇 년생? 01년이요. 25살 신사생 뱅기네? 네. 맞아요. 전체적인 올해 운세 궁금한데 그 중에서 가장 궁금한 게 취업운이랑 학업운이거든요 제가 그래서 일을 그만두고 이제 조금 취업을 위해서 아예 새로운 거를 공부할 건데 이 길이 저한테 맞는지 이게 조금 궁금해서요 어 그러면 새로운 이거니까 이제 한국을 공부하려니 어떤 그... 약간 IT 쪽이에요. 네, 근데 아예 모르는 상태에서 새롭게 아예 공부하는 거라서요. 그래서 이런 거에 대한 약간 되게 겁이 좀 생기고 그리고 이제 또 하고 싶은 게 있는데 글 쓰는 일을 조금 해보고 싶었거든요. 아 니는 글 쓰는 일은 진짜 니가 진미적으로도 해야 된다 아 그래요? 그래서 이게 잘할지 어 니는 글쓰는지는 꼭 해야 된다 잘할 수 있다 어 니는 잘할 수 있어 얼마든지. 업으로 삼을 정도로요? 그런 재능을 가지고 있잖아. 글 쓰는 거. 그래서 이게 취업을 이제 준비하고 공부를 하려고 제대로 하는데 취업을 그 이제 공부가 반년 정도 뒤에 끝나거든요. 이때 취업도 괜찮게 잘할 수 있을까요? 올해는 네가 취업으로 들어와 있다 이보면은 올해는 주문도 들어와 있어야 괜찮아. 아 그래요? 어, 어, 어. 상반기를 가면 쏙 좋고 9월달이 되면 굉장히 너무 좋은 걸 알겠습니다. 아, 네. 제가 이제 공부가 한 10월달쯤에 끝나거든요. 아, 9월달이 굉장히 좋네요 아 9월이 좋아요? 응 그래서 제가 이제 좀 돈 욕심이 좀 많아가지고 지금 일을 이제 그만두고 공부를 하다 보니까 일을 안 했을 때 공부하기 좀 되게 불안하고 수중에 돈이 없는 걸 안 좋아해서 혹시 취업이 된다고 하면은 재물운이 좀 괜찮을까요? 괜찮아 올해는 재물이 좀 음 아, 돈 들어와요? 아 그렇군요. 돈은 괜찮게 들어와서 쓸 수 있다. 아 그렇구나. 혹시 연애 운도 

In [ ]:
# chunks를 CSV 파일로 저장
start_end_text = []
for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("calling01.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.00,11.00,Музыка.
1,11.00,20.00,"상담사와 연결되었습니다. 어? 여보세요 안녕하세요 이름 말씀드리면 될까요? 네, ..."
2,20.00,22.00,"네, 이름 말씀드리면 될까요?"
3,22.00,24.00,"네, 생년월일 말씀해주세요."
4,24.00,26.00,"네, 11월 6일이요."
...,...,...,...
160,497.00,499.00,"응, 응. 그래."
161,499.00,502.00,"아, 네. 이도면 된 것 같아요"
162,504.00,506.02,감사합니다 아 그래 그래 감사합니다.
163,507.60,508.02,어 그래 그래 어.
